# Integration algorithm used to integrate $\mathbf{F}$ in the Crystal Plasticity Model

### Import necessary modules and libraries

In [1]:
import numpy as np
%run fourthOrderTensor.py  #this file contains the functions to create and operate on 4th order elastic tensor.
%run createSchmid.py  # this file contains the create schimd function. Don't change this file
%run updateHardness_vocekocks.py

### State class

First lets define a class that contains the material state information

In [2]:
class state:
    def __init__(self,sigma, defGrad, defGrad_elastic, defGrad_plastic, hardness, pressure):
        self.sigma = sigma
        self.defGrad = defGrad
        self.defGrad_elastic = defGrad_elastic
        self.defGrad_plastic = defGrad_plastic
        self.hardness = hardness
        self.pressure = pressure

## Functions to be used in the calculations

### Function to perform elastic stress calculations
- Right Cauchy green tensor,
$\mathbf{C}^{e}_{n+1}$ = $\mathbf{F}^{e^{T}}_{n+1}$*.$\mathbf{F}^e_{n+1}$
-  Green strain tensor,
$\mathbf{E}^e = \frac{1}{2}\left(\mathbf{C}^e - \mathbf{I} \right)$
- Second Piola Kirchhoff stress,
$\mathbf{S} = \mathcal{L} : \mathbf{E}^e$
- Resolved shear stress on slip systems,

    **for** (i=0; i<num_slipSystems; i++)
    
    $\tau_i = (\mathbf{C}^e\mathbf{S})\mathbf{P}_i$
    
    **end for;**

In [3]:
def elasticCalculations(num_slip_sys, schmid, elastTensor, elasticDefGrad):
    C_e = np.zeros((3,3))
    E_e = np.zeros((3,3))
    S = np.zeros((3,3))
    tau = np.zeros(num_slip_sys)
    
    C_e = np.matmul(np.transpose(elasticDefGrad),elasticDefGrad)
    E_e = 0.5*(C_e - np.identity(3))
    S = np.tensordot(elastTensor, E_e) 
    for i in range(0, num_slip_sys):
        tau[i] = np.tensordot(C_e*S, schmid[i])
    return (C_e, E_e, S, tau)

### Function to calculate Residual

In [4]:
def compute_residual(current_state, new_state, gamma_dot_0, num_slip_sys, tau, m_param, schmid, dt):
    gamma_dot = np.zeros(num_slip_sys)
    residual = np.zeros((3,3))
    M=current_state.defGrad_plastic*np.linalg.inv(new_state.defGrad_plastic)
    for i in range(0, num_slip_sys):
        # TODO: Check if gamma_dot is blowing up. If yes, implement safe_pow()
        gamma_dot[i] = gamma_dot_0*(tau[i]/new_state.hardness[i])*(np.fabs(tau[i]/new_state.hardness[i])**((1.0/m_param)-1))
        residual += gamma_dot[i]*schmid[i]
    residual += (-1/dt)*(np.identity(3) - M)
    norm_residual  = np.linalg.norm(residual, ord = 'fro')
    return (norm_residual, residual)

### Function to compute Jacobian

* $\mathbf{A} = \mathbf{F}_{n+1}^{p^{-T}}\mathbf{F}_{n+1}^{T}\mathbf{F}_{n+1}\mathbf{F}_{n+1}^{p^{-1}}$
* $\mathbf{G} = \frac{-1}{\Delta t}\mathbf{F}^p_n\mathbf{F}^{p^{-1}}_{n+1}$
* $\dfrac{\partial C^e_{{n+1}_{ab}}}{\partial F^p_{n+1_{kn}}} = -A_{kb}F^{p^{-1}}_{n+1_{na}}-A_{ak}F^{p^{-1}}_{n+1_{nb}}$
* $J_{2_{ijkm}} = G_{ik}F^{p^{-1}}_{n+1_{nj}}$
* $J_1=0$
$\dfrac{\partial S}{\partial F^p_{n+1}} = \frac{1}{2}\mathcal{L}:\dfrac{\partial C^e_{n+1}}{\partial F^p_{n+1}}$
* __for__(i=0; i$<$ num\_slip\_system; i++)

    - $\mathbf{B} = P_i S^T_{n+1}$
    - $\mathbf{D} = \mathbf{C}^{e^T}_{n+1}\mathbf{P}_i$
    - $\dfrac{\partial \tau}{\partial F^p_{n+1}} = B:\dfrac{\partial C^e_{n+1}}{\partial F^p_{n+1}} + D:\dfrac{\partial S}{\partial F^p_{n+1}}$
    - $\dfrac{\partial g}{\partial \tau} = \dfrac{\partial \dot{\gamma}_0}{mg_{n+1_i}} \left| \dfrac{\partial \dot{tau}_{n+1_i}}{g_{n+1_i}} \right|^{\frac{1}{m}-1}$
    - $J_1=J_1 + \dfrac{\partial g}{\partial \tau}\left( \mathbf{P}_i \otimes \dfrac{\partial \tau}{\partial F^p_{n+1}}  \right) $

    __end for;__
* $J=J_1+J_2$
            

In [5]:
def jacobian(current_state, new_state, dt, elastTensor, num_slip_sys, schmid, S, C_e, gamma_dot_0, m_param):
    Fpn = current_state.defGrad_plastic 
    Fnp1 = new_state.defGrad 
    Fpnp1inv = np.linalg.inv(new_state.defGrad_plastic)
    A = np.transpose(Fpnp1inv)*np.transpose(Fnp1)*Fnp1*Fpnp1inv
    G = (-1.0/dt)*Fpn*Fpnp1inv
    dC_dFpnp1 = np.zeros((3,3,3,3))
    J2 = np.zeros((3,3,3,3))
    for i in range(0,3):
        for j in range (0,3):
            for k in range (0,3):
                for n in range(0,3):
                    dC_dFpnp1[i,j,k,n] = -A[k,j]*Fpnp1inv[n,i] - A[i,k]*Fpnp1inv[n,j]
                    J2[i,j,k,n] = G[i,k]*Fpnp1inv[n,j]

    dS_dFpnp1 = np.tensordot(elastTensor, 0.5*dC_dFpnp1) 
    J1 = np.zeros((3,3,3,3))
    for i in range(0, num_slip_sys):
        B = schmid[i]*np.transpose(S)
        D = np.transpose(C_e)*schmid[i]
        dt_dFpnp1 = np.tensordot(B, dC_dFpnp1) + np.tensordot(D, dS_dFpnp1)
        dg_dt = (gamma_dot_0/(m_param*new_state.hardness[i]))*(np.fabs(tau[i]/new_state.hardness[i])**((1.0/m_param)-1))
        J1 += dg_dt*outerProduct(schmid[i], dt_dFpnp1)
    J = AsTMatrix(J1)+AsTMatrix(J2)
    return J

## Calculations start from here:

### Input the following:
- Material properties ($C_{ij}$,$m$, $\dot{\gamma_0}$)
- Orientation of the crystal, ($\phi_1$, $\Phi$, $\phi_2$)
- Deformation gradient at the end of the time step, $\mathbf{F}$
- Time step, $\Delta t$
- Change in pressure, $\Delta p$
- Equilibrium state, current_state
- Trial state, new_state

In [6]:
########################################
# Material Properties
C11 = 114290.0 ;
C33 = 157580.0 ;
C12 =  79870.0 ;
C13 =  77326.9 ;
C44 =  24030.0 ;
C66 = 0.5*(C11 - C12);
m_param = 0.38;
g_0 =  13250.0;
gamma_dot_0 = 1.0;
G_0 = 120.0;
g_s_0 = 250.0;
gamma_dot_s = 5.0e+10;
omega = 0.0;
a_bcc = 3.196;
a_hcp = 2.943 ;
c_hcp = 4.68;
aa_ab=a_hcp/a_bcc;
ca_ab=c_hcp/a_bcc;
########################################
elastTensor = ElastTensor(C11, C12, C13, C33, C44, C66)  # Get 4th order elastic tensor
(num_slip_sys, schmid) = createSchimid("hcp") #create schmid tensor

In [7]:
# Change to desired orientation
phi1 = 0
Phi = 0
phi2 = 0

# Rotation matrix R0 = R(phi1)*R(Phi)*R(phi2)
R0 = np.array([[np.cos(phi1)*np.cos(phi2)-np.sin(phi1)*np.sin(phi2)*np.cos(Phi), np.sin(phi1)*np.cos(phi2)+np.cos(phi1)*np.sin(phi2)*np.cos(Phi), np.sin(phi2)*np.sin(Phi)],
              [-np.cos(phi1)*np.sin(phi2)-np.sin(phi1)*np.cos(phi2)*np.cos(Phi), -np.sin(phi1)*np.sin(phi2)+np.cos(phi1)*np.cos(phi2)*np.cos(Phi), np.cos(phi2)*np.sin(Phi)],
              [np.sin(phi1)*np.sin(Phi), -np.cos(phi1)*np.sin(Phi), np.cos(Phi)]])

In [8]:
# change to a desired input def grad (in numpy array format)
idefGrad = np.identity(3) 

In [9]:
# change to desired timestep pressure change
dt = 0.1
dp = 0

In [10]:
# change these to the desired current state
sigma = np.zeros((3,3))
defGrad = np.identity(3)
defGrad_elastic = np.identity(3)
defGrad_plastic = np.identity(3)
hardness = np.ones(num_slip_sys)
pressure = 0
# Now, using these values construct the current_state object
current_state = state(sigma, defGrad, defGrad_elastic, defGrad_plastic, hardness, pressure)
new_state = current_state;

###  Set max iterations and tolerances

In [11]:
max_hardness_iter = 10
max_residual_iter = 100
L2norm_tolerance = 1e-5
hardness_change_tolerance = 1e-6

### Initialize

In [12]:
hardness_error = 0.0
L2norm = 0.0
L2norm_old = 0.0
dFp = np.zeros(9)
Fp_lastgood = np.identity(3)

new_state.pressure = current_state.pressure + dp # Set new state's presssure

numSteps = 4;  # number of steps over which Ft is applied
Ft = np.array([[aa_ab*2/np.sqrt(3), aa_ab*np.sqrt(6)/12, 0],[ 0, aa_ab*3*np.sqrt(2)/4, 0],[ 0, 0, ca_ab/np.sqrt(2)]]);
Ft_s =np.array([np.linspace(1, Ft[0,0],numSteps), np.linspace(0, Ft[0,1],numSteps), np.zeros(numSteps),np.zeros(numSteps), np.linspace(1, Ft[1,1],numSteps),  np.zeros(numSteps),np.zeros(numSteps),np.zeros(numSteps), np.linspace(1, Ft[2,2],numSteps)]).T
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print(Ft_s)

[[ 1.000  0.000  0.000  0.000  1.000  0.000  0.000  0.000  1.000]
 [ 1.021  0.063  0.000  0.000  0.992  0.000  0.000  0.000  1.012]
 [ 1.042  0.125  0.000  0.000  0.984  0.000  0.000  0.000  1.024]
 [ 1.063  0.188  0.000  0.000  0.977  0.000  0.000  0.000  1.035]]


### Start the calculation loops

In [13]:
# Main loop
for nsteps in range(0,numSteps):

    defGrad_transform = Ft_s[nsteps,:].reshape((3,3))
    new_state.defGrad = idefGrad*R0*np.linalg.inv(defGrad_transform)
    # start hardness loop
    for hardness_iter in range(0,max_hardness_iter):

        #hardness convergence check
        if hardness_iter==max_hardness_iter:
            print("Max hardness iteration reached!")
            break
        # start residual loop
        residual_not_converged = False
        for residual_iter in range(0, max_residual_iter):

            # residual convergence check
            if residual_iter==max_residual_iter:
                residual_not_converged = True
                break

            # Make initial guess
            if residual_iter == 0 and hardness_iter == 0:
                new_state.defGrad_elastic = current_state.defGrad_elastic
                new_state.defGrad_plastic = np.matmul(np.linalg.inv(new_state.defGrad_elastic), new_state.defGrad)
                new_state.hardness = current_state.hardness

            # elastic stress calculations
            (C_e, E_e, S, tau) = elasticCalculations(num_slip_sys, schmid, elastTensor, new_state.defGrad_elastic)

            # compute residual
            (L2norm, residual) = compute_residual(current_state, new_state, gamma_dot_0, num_slip_sys, tau, m_param, schmid, dt)

            # convergence check
            if L2norm < L2norm_tolerance:            
                break
            elif residual_iter > 0 and L2norm > L2norm_old:
                new_state.defGrad_plastic = Fp_lastgood
                new_state.defGrad_elastic = new_state.defGrad*np.linalg.inv(new_state.defGrad_plastic)    
                dFp = 0.5*dFp
            else:
                # Apply dFp
                dFp_matrix = dFp.reshape(3,3)
                new_state.defGrad_plastic += dFp_matrix
                if np.linalg.det(new_state.defGrad_plastic) > 1.1 or np.linalg.det(new_state.defGrad_plastic) < 0.9:
                    print("Questionable det(new_state.defGrad_plastic)1: %d\n", np.linalg.det(new_state.defGrad_plastic));
                
                new_state.defGrad_elastic = new_state.defGrad*np.linalg.inv(new_state.defGrad_plastic)
                # proceed to solve if residual is decreasing or if it is the first step
                L2norm_old = L2norm
                Fp_lastgood = new_state.defGrad_plastic        

            # Compute Jacobian
            J = jacobian(current_state, new_state, dt, elastTensor, num_slip_sys, schmid, S, C_e, gamma_dot_0, m_param)       

            residualVector = residual.flatten()

            # Solve
            dFp =  np.linalg.solve(J, -1.0*residualVector)  
            # TODO: add try catch for the solution dFp


            # Apply dFp
            dFp_matrix = dFp.reshape(3,3)
            new_state.defGrad_plastic += dFp_matrix
            if np.linalg.det(new_state.defGrad_plastic) > 1.1 or np.linalg.det(new_state.defGrad_plastic) < 0.9:
                    print("Questionable det(new_state.defGrad_plastic)1: %d\n", np.linalg.det(new_state.defGrad_plastic));
            new_state.defGrad_elastic = new_state.defGrad*np.linalg.inv(new_state.defGrad_plastic)

            if residual_not_converged:
                print("Max residual iteration reached!")
                break

        #update hardness
#         hardness_error = updateHardness_vocekocks(current_state, new_state, dt, m_param, g_0, gamma_dot_0, G_0, g_s_0, gamma_dot_s, omega, num_slip_sys, schmid, elastTensor)
        # For HCP lattic, no hardness:
        hardness_error = 0.0

        # hardness convergence check
        if hardness_error < hardness_change_tolerance:
            break


    #Scale Fp, det(Fp) = 1

    scale = np.linalg.det(new_state.defGrad_plastic)**(1.0/3.0)
    new_state.defGrad_plastic /= scale
    new_state.defGrad_elastic = new_state.defGrad*np.linalg.inv(new_state.defGrad_plastic)    

    # recompute elastic quantities
    (C_e, E_e, S, tau) = elasticCalculations(num_slip_sys, schmid, elastTensor, new_state.defGrad_elastic)

    cauchy = new_state.defGrad_elastic*S*np.transpose(new_state.defGrad_elastic)/np.linalg.det(new_state.defGrad_elastic)
    deviatoric = cauchy - (np.trace(cauchy)/3.0)*np.identity(3)

    stress = deviatoric + new_state.pressure*np.identity(3)
    current_state = new_state
print(stress)

[[ 328.796  0.000  0.000]
 [ 0.000  328.796  0.000]
 [ 0.000  0.000 -657.592]]
